<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/LocalMinimaEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from typing import Tuple

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image

In [29]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/zw2788/LocalMinimaConstruction/main/Ex1.csv")

data.head()

,x_2dvec,y
0,"[2.8,0.4]",1
1,"[3.1,4.3]",1
2,"[0.1,-3.4]",1
3,"[-4.2,-3.3]",1
4,"[-0.5,0.2]",1


In [30]:
features = [
    "x_2dvec",

]
label = "y"

# train test split
X_raw,  Y = data[features].values, data[label].values

#convert string to array
X_raw = np.array([eval(s[0]) for s in X_raw])

# Standardize the input
# Leave blank to match the example in paper

# formatting
Y = Y.reshape((-1, 1))
print(X_raw)
print(Y)
print(X_raw.shape[0])

[[ 2.8  0.4]
 [ 3.1  4.3]
 [ 0.1 -3.4]
 [-4.2 -3.3]
 [-0.5  0.2]
 [-2.7 -0.4]
 [-3.  -4.3]
 [-0.1  3.4]
 [ 4.2  3.2]
 [ 0.4 -0.1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
10


Neural Structure:
$M((x_0, x_1)) = \sigma(v_0σ(w_{0,0}x_0 + w_{0,1}x_1 + b_0) + v_1σ(w_{1,0}x_0 + w_{1,1}x_1 + b_1) + c) $

In [31]:
Image(url='https://github.com/zw2788/LocalMinimaConstruction/blob/main/221Sigmoid.png?raw=true')


In [32]:
def sigmoid(x):
    """Calculates sigmoid function."""
    return 1. / (1 + np.exp(-x))

# parameters between input layer and the hidden layer
W_0 = np.random.normal(size=(2, X_raw.shape[1]))
print(f"Shape of W_0 is {W_0.shape}")
print(W_0)
print(W_0.shape[1])
b = np.random.normal(size=(2, 1))
print(f"Shape of b_1 is {b.shape}")

# parameters between hidden layer and output layer
V_0 = np.random.normal(size=(1, W_0.shape[1]))
c = np.random.normal(size=(1, 1))

# calculate the forward propagation
Z_1 = X_raw @ W_0.T
print(f"\nShape of Z_1 is {Z_1.shape}")
print("Samples for Z_1:")
print(Z_1[:5])

A_1 = sigmoid(Z_1 + b.T)
print(f"Shape of A_1 is {A_1.shape}")
print("Samples for A_1:")
print(A_1[:5])

Z_2 = A_1 @ V_0.T
print(f"\nShape of Z_2 is {Z_2.shape}")
print("Samples for Z_2:")
print(Z_2[:5])

A_2 = Y_hat = sigmoid(Z_2 + c.T)
print(f"Shape of A_2 is {A_2.shape}")
print("Samples for A_2:")
print(A_2[:5])



Shape of W_0 is (2, 2)
[[ 0.32220473 -1.27107645]
 [ 0.07631443  1.06754938]]
2
Shape of b_1 is (2, 1)

Shape of Z_1 is (10, 2)
Samples for Z_1:
[[ 0.39374267  0.64070017]
 [-4.46679407  4.82703706]
 [ 4.35388041 -3.62203643]
 [ 2.84129241 -3.84343356]
 [-0.41531766  0.17535266]]
Shape of A_1 is (10, 2)
Samples for A_1:
[[0.77208989 0.75861431]
 [0.02557114 0.99518616]
 [0.99440509 0.04238549]
 [0.97510066 0.03425603]
 [0.60134906 0.66368289]]

Shape of Z_2 is (10, 1)
Samples for Z_2:
[[-0.71738025]
 [-0.27242248]
 [-0.6843528 ]
 [-0.66919447]
 [-0.57740689]]
Shape of A_2 is (10, 1)
Samples for A_2:
[[0.3207757 ]
 [0.42427422]
 [0.32801387]
 [0.33136375]
 [0.35200539]]


In [33]:
def forward_prop(
    X_raw: np.array,
    W_0: np.array,
    b: np.array,
    V_0: np.array,
    c: np.array,
) -> Tuple:
    """Performs the forward propagation of the given NN."""
    # Note the NN structure is passed in from outside.
    Z_1 = X_raw @ W_0.T
    A_1 = sigmoid(Z_1 + b.T)

    Z_2 = A_1 @ V_0.T
    A_2 = sigmoid(Z_2 + c.T)

    return A_2, Z_2, A_1, Z_1

Y_hat, _, _, _ = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)

In [34]:
def derivatives_by_backprop(
    X_raw: np.array,
    Y: np.array,
    W_0: np.array,
    b: np.array,
    V_0: np.array,
    c: np.array,
) -> Tuple:
    """Calculates the derivatives of the parameters by backforward propagation.

    Here we assume it is a binary classification problem, with sigmoid activation functions.
    """
    # forward propagation
    dW_0, db, dV_0, dc = 0, 0, 0, 0
    Y_hat, Z_2, A_1, Z_1 = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)
    n = len(Y_hat)

    loss = -np.mean(np.multiply(Y, np.log(Y_hat)) + np.multiply(1 - Y, np.log(1 - Y_hat)))

    dZ_2 = Y_hat - Y
    dV_0 = dZ_2.T @ A_1 / n
    dc = np.mean(dZ_2.T, axis=1, keepdims=True)

    dZ_1 = np.multiply(dZ_2 @ V_0, np.multiply(A_1, 1 - A_1))
    dW_0 = (dZ_1.T @ X_raw) / n
    db = np.mean(dZ_1.T, axis=1, keepdims=True)

    return dW_0, db, dV_0, dc, loss

dW_0, db, dV_0, dc, loss = derivatives_by_backprop(X_raw=X_raw, Y=Y, W_0=W_0, b=b, V_0=V_0, c=c)

In [35]:
def gradient_descent(
    X_raw: np.array,
    Y: np.array,
    W_0_init: np.array,
    b_init: np.array,
    V_0_init: np.array,
    c_init: np.array,
    learning_rate: float = 0.01,
    epsilon: float = 1e-6,
    verbose: bool = False,
) -> Tuple:
    """Runs gradient descent to fit the NN via backprop."""
    W_0 = W_0_init
    b = b_init
    V_0 = V_0_init
    c = c_init
    losses = [float("inf"), ]
    roc_auc_scores = [0.5, ]

    diff_in_loss = float("inf")
    iteration = 0
    while abs(diff_in_loss) > epsilon:
        iteration += 1
        dW_0, db, dV_0, dc, loss = derivatives_by_backprop(
            X_raw=X_raw, Y=Y, W_0=W_0, b=b, V_0=V_0, c=c
        )

        W_0 -= learning_rate * dW_0
        b  -= learning_rate * db
        V_0 -= learning_rate * dV_0
        c  -= learning_rate * dc

        losses.append(loss)
        diff_in_loss = losses[-1] - losses[-2]

        Y_hat, _, _, _ = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)
        roc_auc = roc_auc_score(y_true=Y, y_score=Y_hat)
        roc_auc_scores.append(roc_auc)

        if verbose and iteration % 10 == 0:
            print(loss, roc_auc)
    return W_0, b, V_0, c, losses

In [41]:
# parameters for the first layer
W_0_init = np.array([[1.06*(1+0.001*np.random.choice([-1,1])),-0.037*(1+0.001*np.random.choice([-1,1]))],[-0.056*(1+0.001*np.random.choice([-1,1])),1.095*(1+0.001*np.random.choice([-1,1]))]])
b_init = np.array([[-0.051*(1+0.001*np.random.choice([-1,1]))],[-0.0689*(1+0.001*np.random.choice([-1,1]))]])

# parameters for the second layer

V_0_init = np.array([[3.769*(1+0.001*np.random.choice([-1,1])),-3.72*(1+0.001*np.random.choice([-1,1]))]])
c_init = np.array([[-0.0148*(1+0.001*np.random.choice([-1,1]))]])

#random initial points
#np.random.seed(42)
#W_0_init = np.random.normal(size=(2, X_raw.shape[1]))
#b_init  = np.random.normal(size=(2, 1))
#V_0_init = np.random.normal(size=(1, W_0.shape[1]))
#c_init  = np.random.normal(size=(1, 1))

#initial points from all zero
#W_0_init = np.array([[0.1,0.1],[0.1,0.1]])
#b_init = np.array([[0.1],[0.1]])
#V_0_init = np.array([[0.1,0.1]])
#c_init = np.array([[0.1]])

W_0, b, V_0, c, losses = gradient_descent(
    X_raw=X_raw,
    Y=Y,
    W_0_init=W_0_init,
    b_init =b_init,
    V_0_init=V_0_init,
    c_init =c_init,
    learning_rate=0.05,
    epsilon=1e-8,
    verbose=True,
)
print(W_0.T)
print(b)
print(V_0.T)
print(c)
print(losses)

0.577738976830658 0.64
0.5777386915522563 0.64
0.5777384955501235 0.64
0.5777383595731831 0.64
[[ 1.05915871 -0.05625444]
 [-0.03706663  1.09392742]]
[[-0.0505957 ]
 [-0.06918686]]
[[ 3.76597509]
 [-3.72305822]]
[[-0.01387154]]
[inf, 0.5777393460321049, 0.5777392983587557, 0.5777392525084735, 0.577739208408196, 0.5777391659878883, 0.5777391251804123, 0.5777390859214021, 0.5777390481491478, 0.5777390118044785, 0.577738976830658, 0.5777389431732781, 0.5777389107801616, 0.5777388796012671, 0.577738849588598, 0.5777388206961177, 0.5777387928796649, 0.5777387660968765, 0.5777387403071106, 0.5777387154713745, 0.5777386915522563, 0.5777386685138585, 0.5777386463217338, 0.5777386249428256, 0.5777386043454099, 0.5777385844990397, 0.577738565374492, 0.5777385469437166, 0.5777385291797892, 0.5777385120568624, 0.5777384955501235, 0.5777384796357501, 0.577738464290871, 0.5777384494935254, 0.5777384352226272, 0.5777384214579279, 0.5777384081799826, 0.5777383953701178, 0.5777383830103995, 0.577738371

Remarks:
1.Not a global minimum. (details see deepmind paper's proof.)
2.

What a few first failed attempts made us realize was, that the nature of a successful example would
have to be both geometric and analytic at the same time. What “deadlocks” a sigmoid-based neural
network is not only the geometric configuration of the points, but also the very precise cross-ratios of
distances between them. The successful construction of the presented example was a combination of
studying the failed attempts generated by a guesswork and then trying to block the “escape routes”
with a gradient descent in the data space. Once a “close enough” configuration of points was deduced, the gradient descent was applied to the datapoints (in the data space) in order to minimize the
length of the gradient in the weights space of the loss function. This procedure modifies the dataset
in such a way that the (fixed) set of weights becomes a critical point of the error surface, but starting
from a randomly chosen dataset almost surely produces a saddle point, instead of a minimum. Using
the “close enough” configuration yields a higher chance of finding a true local minimum.
